In [ ]:
E_L = 200 #GPa
E_L = 200e9 #Pa
E_T = 40 #GPa
E_T = 40e9 #Pa
nu_LT = 0.2
# nu_TL = E_T/E_L*nu_LT
nu_TL = E_T/E_L*nu_LT
G_LT = 30 #GPa
G_LT = 30e9 #Pa
t = 0.1 #mm
t = 0.1e-3 #m
angle_list = [0, 45, 90, -45]
repetitions = 2

nu_TL

0.04000000000000001

# <center> A <center>

In [ ]:
#A laminate is consisting of a ply stack of [0/45/90/-45]2. Material properties of a ply is given right. The ply thickness is 0.1 mm
#a) Calculate the A matrix, D matrix, and B matrix of this laminate.
import numpy as np
import sympy as sym
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Q matrix
Q = [[E_L/(1-nu_LT*nu_TL), E_L*nu_TL/(1-nu_LT*nu_TL), 0],
     [E_L*nu_TL/(1-nu_LT*nu_TL), E_T/(1-nu_LT*nu_TL), 0],
     [0, 0, G_LT]]
display(Q)
T_sigma = lambda theta: [[np.cos(theta)**2, np.sin(theta)**2, 2*np.cos(theta)*np.sin(theta)],
                   [np.sin(theta)**2, np.cos(theta)**2, -2*np.cos(theta)*np.sin(theta)],
                   [-np.cos(theta)*np.sin(theta), np.cos(theta)*np.sin(theta), np.cos(theta)**2-np.sin(theta)**2]]
T_epsilon = lambda theta: [[np.cos(theta)**2, np.sin(theta)**2, np.cos(theta)*np.sin(theta)],
                     [np.sin(theta)**2, np.cos(theta)**2, -np.cos(theta)*np.sin(theta)],
                     [-2*np.cos(theta)*np.sin(theta), 2*np.cos(theta)*np.sin(theta), np.cos(theta)**2-np.sin(theta)**2]]
def Q_bar(theta):
    T_sigma_ = T_sigma(theta)
    T_epsilon_ = T_epsilon(theta)
    return np.dot(np.dot(np.linalg.inv(T_sigma_), Q), T_epsilon_)

# Convert to radians
angle_list = [angle*np.pi/180 for angle in angle_list]
Q_list = []
for angle in angle_list:
    print(f"Q_bar({angle})")
    display(sym.Matrix(Q_bar(angle)))
    Q_list.append(Q_bar(angle))
    print()

display(sym.Matrix(Q_list))


[[201612903225.80646, 8064516129.03226, 0],
 [8064516129.03226, 40322580645.16129, 0],
 [0, 0, 30000000000.0]]

Q_bar(0.0)


Matrix([
[201612903225.806, 8064516129.03226,             0],
[8064516129.03226, 40322580645.1613,             0],
[               0,                0, 30000000000.0]])


Q_bar(0.7853981633974483)


Matrix([
[94516129032.2581, 34516129032.2581, 40322580645.1613],
[34516129032.2581, 94516129032.2581, 40322580645.1613],
[40322580645.1613, 40322580645.1613, 56451612903.2258]])


Q_bar(1.5707963267948966)


Matrix([
[   40322580645.1613,    8064516129.03226, 1.69870362462375e-6],
[   8064516129.03226,    201612903225.806, 8.17748023946781e-6],
[1.69870362462375e-6, 8.17748023946781e-6,       30000000000.0]])


Q_bar(-0.7853981633974483)


Matrix([
[ 94516129032.2581,  34516129032.2581, -40322580645.1613],
[ 34516129032.2581,  94516129032.2581, -40322580645.1613],
[-40322580645.1613, -40322580645.1613,  56451612903.2258]])

Matrix([
[                [201612903225.806, 8064516129.03226, 0.0],                 [8064516129.03226, 40322580645.1613, 0.0],                                 [0.0, 0.0, 30000000000.0]],
[   [94516129032.2581, 34516129032.2581, 40322580645.1613],    [34516129032.2581, 94516129032.2581, 40322580645.1613],    [40322580645.1613, 40322580645.1613, 56451612903.2258]],
[[40322580645.1613, 8064516129.03226, 1.69870362462375e-6], [8064516129.03226, 201612903225.806, 8.17748023946781e-6], [1.69870362462375e-6, 8.17748023946781e-6, 30000000000.0]],
[  [94516129032.2581, 34516129032.2581, -40322580645.1613],   [34516129032.2581, 94516129032.2581, -40322580645.1613],  [-40322580645.1613, -40322580645.1613, 56451612903.2258]]])

In [ ]:
# A matrix
A = 2*t*(sum(Q_list))
display(sym.Matrix(A))

Matrix([
[86193548.3870968, 17032258.0645161,                0],
[17032258.0645161, 86193548.3870968, 1.52587890625e-9],
[               0, 1.52587890625e-9, 34580645.1612903]])

In [ ]:
def BD_func(symmetric=False, repetitions=1, t=1):
    D = 0
    B = 0
    m=0
    if not symmetric:
        for i in np.arange(repetitions/2 * len(angle_list), -repetitions/2 * len(angle_list), -1):
            B += ((t*i)**2-(t*(i-1))**2)/2 * Q_list[m]
            D += ((t*i)**3-(t*(i-1))**3)/3 * Q_list[m]
            m += 1
            if m == len(angle_list):
                m = 0
        
    if symmetric:
        B = np.zeros((3, 3))
        repetitions = repetitions*2
        for i in np.arange(len(angle_list), 0, -1):
            D += ((t*i)**3-(t*(i-1))**3)/3 * Q_list[m]
            m += 1
            if m == len(angle_list):
                m = 0

        if m == 0:
                m = len(angle_list)-1
                
        for i in np.arange(0, -len(angle_list), -1):
            D += ((t*i)**3-(t*(i-1))**3)/3 * Q_list[m]
            m -= 1
            if m == -len(angle_list):
                m = 0
                
    return B, D

B, D = BD_func(symmetric=False, repetitions=repetitions, t=t)
display(sym.Matrix(B), sym.Matrix(D))

Matrix([
[ 3754.83870967742, -529.032258064516,  1612.90322580645],
[-529.032258064516, -2696.77419354839,  1612.90322580645],
[ 1612.90322580645,  1612.90322580645, -529.032258064516]])

Matrix([
[  4.91956989247312,  0.908387096774194, -0.161290322580645],
[ 0.908387096774194,   4.27440860215054, -0.161290322580645],
[-0.161290322580645, -0.161290322580645,   1.84430107526882]])

# <center> B <center>

In [ ]:
# b) Solve for the strain of the laminate, when Mx of 1000 [N] is applied. Where in the plate experiences the largest strains?
def find_epsilon_kappa(N_x=0, N_y=0, N_z=0, M_x=0, M_y=0, M_z=0):
    N_matrix = np.transpose([N_x, N_y, N_z])
    M_matrix = np.transpose([M_x, M_y, M_z])
    N_M_matrix = np.block([N_matrix, M_matrix])
    ABDB_matrix = np.block([[A, B], [B, D]])
    epsilon_kappa_matrix = np.linalg.inv(ABDB_matrix) @ N_M_matrix
    return epsilon_kappa_matrix

epsilon_kappa_matrix = find_epsilon_kappa(M_x = 1000)
sym.Matrix(epsilon_kappa_matrix)


Matrix([
[ -0.0107234325020912],
[  0.0017197509196674],
[-0.00803541094558697],
[    223.060228613911],
[   -43.8126949951916],
[    21.2448870284602]])

In [ ]:
epsilom_matrix = epsilon_kappa_matrix[:3]
kappa_matrix = epsilon_kappa_matrix[3:]
display(sym.Matrix(epsilom_matrix), sym.Matrix(kappa_matrix))

Matrix([
[ -0.0107234325020912],
[  0.0017197509196674],
[-0.00803541094558697]])

Matrix([
[ 223.060228613911],
[-43.8126949951916],
[ 21.2448870284602]])

In [ ]:
print("Top")
display(sym.Matrix(epsilom_matrix) + 3*t*sym.Matrix(kappa_matrix))
print("Bottom")
display(sym.Matrix(epsilom_matrix) - 3*t*sym.Matrix(kappa_matrix))

Top


Matrix([
[  0.0561946360820821],
[ -0.0114240575788901],
[-0.00166194483704892]])

Bottom


Matrix([
[-0.0776415010862646],
[ 0.0148635594182249],
[ -0.014408877054125]])

When multiplying the $k$ matrix by $\pm 0.0003$ this is what we obtain. These are the maximum stresses at the top and bottom surfaces of our plate. We can notice that the stress along the x will be greatest in magnitude at the bottom of the plate, while for the stress in the y direction the magnitude will be greatest at the bottom of the plate. This also applies to $\epsilon_3$, as in the largest magnitude is at the bottom.

# <center> C <center>

In [ ]:
# Strain in each ply
Q_list = []
for angle in angle_list:
    print(f"Q_bar({angle})")
    display(sym.Matrix(Q_bar(angle)))
    Q_list.append(Q_bar(angle))
    print()


Q_bar(0.0)


Matrix([
[201612903225.806, 8064516129.03226,             0],
[8064516129.03226, 40322580645.1613,             0],
[               0,                0, 30000000000.0]])


Q_bar(0.7853981633974483)


Matrix([
[94516129032.2581, 34516129032.2581, 40322580645.1613],
[34516129032.2581, 94516129032.2581, 40322580645.1613],
[40322580645.1613, 40322580645.1613, 56451612903.2258]])


Q_bar(1.5707963267948966)


Matrix([
[   40322580645.1613,    8064516129.03226, 1.69870362462375e-6],
[   8064516129.03226,    201612903225.806, 8.17748023946781e-6],
[1.69870362462375e-6, 8.17748023946781e-6,       30000000000.0]])


Q_bar(-0.7853981633974483)


Matrix([
[ 94516129032.2581,  34516129032.2581, -40322580645.1613],
[ 34516129032.2581,  94516129032.2581, -40322580645.1613],
[-40322580645.1613, -40322580645.1613,  56451612903.2258]])

In [ ]:
A = 2*t*(sum(Q_list))
B, D = BD_func(symmetric=True, repetitions=1, t=t)
display(sym.Matrix(B), sym.Matrix(D))


Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

Matrix([
[ 6.42150537634409, 0.696774193548387, 0.483870967741936],
[0.696774193548387,  3.19569892473118, 0.483870967741936],
[0.483870967741936, 0.483870967741936,  1.63268817204301]])

#### B=0 because of symmetry

In [ ]:
M_x = 1000

epsilon_kappa_matrix = find_epsilon_kappa(M_x=M_x)
kappa_matrix = epsilon_kappa_matrix[3:]
sym.Matrix(kappa_matrix)

Matrix([
[ 161.870484067296],
[-29.3466191160564],
[-39.2753814726409]])

#### If a plate is symmetric it will not experience shear forces, but only bending and twisting. While the asymmetric plate will experience all the forces. The asymmetric plate will also be elongated and sheared.